In [1]:
import numpy as np
import torch
import torch.nn as nn

In [2]:
datax = [
    [[0.1517, 0.5443, 0.1664, 0.2968, 0.1047, 0.2303, 0.7809, 0.8742]],
    [[0.6482, 0.8415, 0.3437, 0.5621, 0.8318, 0.5978, 0.2179, 0.5826]],
    [[0.0959, 0.5683, 0.8281, 0.2220, 0.7972, 0.3602, 0.9269, 0.3220]],
    [[0.2017, 0.4027, 0.9019, 0.3093, 0.2855, 0.0226, 0.3115, 0.5141]],
]

# data shape
batch = 4
length = 8

# conv shape
k_size = 3
chan_in = 1
chan_out = 2

params = np.random.rand(k_size, chan_in, chan_out)


In [3]:
np_input = np.array(datax).reshape(batch, length, chan_in)
th_input = torch.Tensor(datax)

In [4]:
from lamp.conv import Conv1D

m1 = Conv1D(k_size, chan_in, chan_out)
output = m1.forward(np_input)
np.round_(output.reshape(batch, chan_out, -1), decimals=4)

array([[[-0.0065,  0.0458,  0.0321,  0.025 ,  0.0034,  0.0268],
        [ 0.014 ,  0.0148, -0.0321,  0.0207, -0.0378,  0.0664]],

       [[ 0.0237,  0.0799,  0.0429,  0.0454, -0.0204,  0.0518],
        [ 0.0043,  0.0774,  0.0521,  0.0648,  0.0225,  0.0303]],

       [[-0.043 ,  0.0469,  0.0224,  0.077 ,  0.0335,  0.0358],
        [-0.015 ,  0.0672,  0.0214,  0.046 , -0.0036,  0.0802]],

       [[-0.0329,  0.0363,  0.0015,  0.0792,  0.0621,  0.0439],
        [ 0.0206,  0.029 ,  0.0111,  0.0082, -0.0291,  0.0251]]])

In [5]:
m2 = nn.Conv1d(chan_in, chan_out, k_size)
output = m2(th_input)
output

tensor([[[-0.3171, -0.5713, -0.3103, -0.4320, -0.5647, -0.6349],
         [-0.2240, -0.4373, -0.3536, -0.4147, -0.0642,  0.1561]],

        [[-0.6062, -0.8102, -0.6805, -0.6719, -0.6511, -0.7118],
         [-0.1813, -0.3421,  0.0160, -0.0472, -0.3895, -0.3079]],

        [[-0.5638, -0.5195, -0.9072, -0.4176, -0.9398, -0.4596],
         [ 0.0930, -0.2155, -0.2829, -0.0605, -0.1618, -0.0684]],

        [[-0.6514, -0.4753, -0.7255, -0.3431, -0.4643, -0.4123],
         [ 0.0273, -0.0970, -0.4957, -0.4381, -0.4081, -0.1239]]],
       grad_fn=<SqueezeBackward1>)

In [6]:
from lamp import MaxPool1D

m3 = MaxPool1D(k_size, stride=1)
output = m3.forward(np_input)
output.reshape(batch, chan_in, -1)

array([[[0.5443, 0.5443, 0.2968, 0.2968, 0.7809, 0.8742]],

       [[0.8415, 0.8415, 0.8318, 0.8318, 0.8318, 0.5978]],

       [[0.8281, 0.8281, 0.8281, 0.7972, 0.9269, 0.9269]],

       [[0.9019, 0.9019, 0.9019, 0.3093, 0.3115, 0.5141]]])

In [7]:
m4 = nn.MaxPool1d(k_size, stride=1)
output = m4(th_input)
output

tensor([[[0.5443, 0.5443, 0.2968, 0.2968, 0.7809, 0.8742]],

        [[0.8415, 0.8415, 0.8318, 0.8318, 0.8318, 0.5978]],

        [[0.8281, 0.8281, 0.8281, 0.7972, 0.9269, 0.9269]],

        [[0.9019, 0.9019, 0.9019, 0.3093, 0.3115, 0.5141]]])

In [8]:
input = m4(th_input)
# With default parameters
m6 = nn.Flatten()
output = m6(input)
output.size(), output

(torch.Size([4, 6]),
 tensor([[0.5443, 0.5443, 0.2968, 0.2968, 0.7809, 0.8742],
         [0.8415, 0.8415, 0.8318, 0.8318, 0.8318, 0.5978],
         [0.8281, 0.8281, 0.8281, 0.7972, 0.9269, 0.9269],
         [0.9019, 0.9019, 0.9019, 0.3093, 0.3115, 0.5141]]))

In [9]:
from lamp import Flatten

input = m3.forward(np_input)

m5 = Flatten()
output = m5.forward(input)
output.shape, output

((4, 6),
 array([[0.5443, 0.5443, 0.2968, 0.2968, 0.7809, 0.8742],
        [0.8415, 0.8415, 0.8318, 0.8318, 0.8318, 0.5978],
        [0.8281, 0.8281, 0.8281, 0.7972, 0.9269, 0.9269],
        [0.9019, 0.9019, 0.9019, 0.3093, 0.3115, 0.5141]]))